In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


# Installing and Importing Packages

First install following packages:
1. [CTGAN](https://pypi.org/project/ctgan/)
2. [SDV](https://pypi.org/project/sdv/)

In [2]:
!pip install ctgan
!pip install sdv

     |████████████████████████████████| 9.5 MB 1.7 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.5 which is incompatible.
pyldavis 3.3.1 requires pandas>=1.2.0, but you have pandas 1.1.4 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.2 which is incompatible.
featuretools 0.25.0 requires pandas<2.0.0,>=1.2.0, but you have pandas 1.1.4 which is incompatible.
     |████████████████████████████████| 88 kB 356 kB/s 
     |████████████████████████████████| 95 kB 1.0 MB/s 
     |████████████████████████████████| 1.1 MB 2.0 MB/s 
     |████████████████████████████████| 5.7 MB 17.7 MB/s 
     |█████████████

Now import required tools:

In [3]:
from sdv.tabular import CTGAN
from sdv.evaluation import evaluate

# Read Original Data

CTAGN is generative neural network which will be trained on `train.csv`. 

In [4]:
train_df = pd.read_csv('/kaggle/input/30-days-of-ml/train.csv')

Drop `id` column because it is not part of original data and is more like index.

In [5]:
train_df.drop(columns=['id'], inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    300000 non-null  object 
 1   cat1    300000 non-null  object 
 2   cat2    300000 non-null  object 
 3   cat3    300000 non-null  object 
 4   cat4    300000 non-null  object 
 5   cat5    300000 non-null  object 
 6   cat6    300000 non-null  object 
 7   cat7    300000 non-null  object 
 8   cat8    300000 non-null  object 
 9   cat9    300000 non-null  object 
 10  cont0   300000 non-null  float64
 11  cont1   300000 non-null  float64
 12  cont2   300000 non-null  float64
 13  cont3   300000 non-null  float64
 14  cont4   300000 non-null  float64
 15  cont5   300000 non-null  float64
 16  cont6   300000 non-null  float64
 17  cont7   300000 non-null  float64
 18  cont8   300000 non-null  float64
 19  cont9   300000 non-null  float64
 20  cont10  300000 non-null  float64
 21  cont11  30

# Train GAN Model

Two important hyper-parameters are `batch_size` and `epoch`.

*Notice that the value that you set on the `batch_size` argument must always be a multiple of 10!*

sklearn will show a lot of annoying warnings so you can turn warnings off. (this is a known [issue](https://github.com/sdv-dev/SDV/issues/568))


In [6]:
import warnings

warnings.filterwarnings('ignore')

In [7]:
batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(train_df)

Epoch 1, Loss G:  0.4254,Loss D:  0.1360
Epoch 2, Loss G:  0.0822,Loss D:  0.0715
Epoch 3, Loss G:  0.0389,Loss D: -0.1512
Epoch 4, Loss G: -1.5512,Loss D: -0.0075
Epoch 5, Loss G: -2.5391,Loss D:  0.1535
Epoch 6, Loss G: -2.5878,Loss D:  0.0653
Epoch 7, Loss G: -2.5074,Loss D: -0.0076
Epoch 8, Loss G: -2.5953,Loss D:  0.1226
Epoch 9, Loss G: -2.2228,Loss D: -0.0504
Epoch 10, Loss G: -2.1559,Loss D:  0.0203
Epoch 11, Loss G: -2.0703,Loss D:  0.1245
Epoch 12, Loss G: -1.4796,Loss D: -0.0329
Epoch 13, Loss G: -1.3505,Loss D: -0.3048
Epoch 14, Loss G: -1.2220,Loss D: -0.1582
Epoch 15, Loss G: -1.9242,Loss D:  0.1401
Epoch 16, Loss G: -1.8132,Loss D: -0.0114
Epoch 17, Loss G: -2.3175,Loss D: -0.0782
Epoch 18, Loss G: -2.0519,Loss D: -0.0104
Epoch 19, Loss G: -2.2142,Loss D:  0.0768
Epoch 20, Loss G: -2.2829,Loss D:  0.0385
Epoch 21, Loss G: -2.7418,Loss D:  0.0257
Epoch 22, Loss G: -2.0307,Loss D:  0.0894
Epoch 23, Loss G: -2.6447,Loss D:  0.0919
Epoch 24, Loss G: -2.3864,Loss D:  0.0598
E

Save your trained model to use later for generating new data.

In [8]:
model.save('ctgan_30daysofml_model.pkl')

Sample new data as follow:

In [9]:
n_generated_data = 1000
generated_df = model.sample(n_generated_data)

Meassure similarity of generated data and the original ones.

In [10]:
score = evaluate(generated_df, train_df.sample(n_generated_data))
score

0.7594668143806097